# Model of work distribution in a flat organization

* no managers; only workers
* no support staff; all infrastructure for communication and tasks works without failure
* no dark patterns

TODO:
* use realistic distributions (e.g., power law) instead of uniform
* instead of searching randomly, search second-order contacts

In [1]:
import random
import numpy
print("numpy", numpy.__version__)
import pandas
print("pandas", pandas.__version__)
import seaborn
print("seaborn", seaborn.__version__)
import matplotlib
print("matplotlib", matplotlib.__version__)
from matplotlib import pyplot as plt
import sys
print(sys.version_info)

numpy 1.23.5
pandas 1.5.2
seaborn 0.12.2
matplotlib 3.6.2
sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0)


In [ ]:
pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

In [ ]:
import lib_simulation

### global variables for simulation

#### user-defined

In [ ]:
import configuration_simCount10_skills2_levels2_duration1_people8_social0_ticks100 as config

### input validation

In [ ]:
if len(config.skill_set_for_people)<1:
    raise Exception("invalid value")
if len(config.skill_set_for_tasks)<1:
    raise Exception("invalid value")
    
if config.max_skill_level_per_person<1:
    raise Exception("invalid value")
if config.max_skill_level_per_task<1:
    raise Exception("invalid value")
if config.max_task_duration_in_ticks<1:
    raise Exception("invalid value")
    
if config.min_number_of_people<1:
    raise Exception("invalid value")
if config.max_number_of_people<1:
    raise Exception("invalid value")
if ((config.min_number_of_people) > (config.max_number_of_people)):
    print(config.min_number_of_people)
    print(config.max_number_of_people)
    print((config.min_number_of_people) > (config.max_number_of_people))
    raise Exception("invalid value")

if config.min_social_circle_size<0:
    raise Exception("invalid value")
if config.max_social_circle_size<0:
    raise Exception("invalid value")
if config.min_social_circle_size>config.max_social_circle_size:
    print(config.min_social_circle_size)
    print(config.max_social_circle_size)
    raise Exception("invalid value")
    
if config.max_ticks_to_simulate<1:
    raise Exception("invalid value")
    
if config.number_of_simultions<1:
    raise Exception("invalid value")

## no parameter sweep, just an ensemble

In [ ]:
number_of_people = config.min_number_of_people
social_circle_size = config.min_social_circle_size

### Initialize Simulation

In [ ]:
sim_dict = {}

for sim_index in range(config.number_of_simultions):
    list_of_people = []
    for person_index in range(number_of_people):
        list_of_people.append(lib_simulation.CreatePerson(person_index, 
                                                          config.skill_set_for_people, 
                                                          config.max_skill_level_per_person))


    tasks_dict = {} # track the generated tasks for post-simulation analysis

    task_id = -1

    # randomly distribute tasks to people
    for index in range(len(list_of_people)):
        #print('person ID',list_of_people[index].unique_id)
        task_id+=1
        tasks_dict = lib_simulation.new_task(task_id,
                                             config.skill_set_for_tasks,
                                             config.max_skill_level_per_task,
                                             config.max_task_duration_in_ticks,
                                             tasks_dict)
        list_of_people[index].backlog_of_tasks.append(tasks_dict[task_id])


    list_of_people, tasks_dict = lib_simulation.simulate(config.skill_set_for_tasks,
                                                         config.max_skill_level_per_task,
                                                         config.max_ticks_to_simulate,
                                                         config.max_task_duration_in_ticks,
                                                         list_of_people,
                                                         tasks_dict)
    
    sim_dict[sim_index] = {'people': list_of_people,
                           'tasks': tasks_dict}

## Post-simulation analysis

# Visualization of the Simulation

In [ ]:
person_time_backlog_dict = {}
for person_index in range(len(list_of_people)):
    
    description=""
    for specialization,skilllevel in list_of_people[person_index].skill_specialization_dict.items():
        description+=specialization+str(skilllevel)+","
    
    person_label = str(person_index)+": "+description[:-1]
    
    person_time_backlog_dict[person_label] = []
    for tick_index, journal_dict in list_of_people[person_index].work_journal_per_tick.items():
        #print(person_id, tick_index, journal_dict['number of tasks in backlog'])
        person_time_backlog_dict[person_label].append(journal_dict['number of tasks in backlog'])

In [ ]:
df_person_time_backlog = pandas.DataFrame(person_time_backlog_dict)
#df_person_time_backlog

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

seaborn.heatmap(df_person_time_backlog)
plt.title('backlog length per person')
plt.gca().set_xlabel('person', fontsize=15);
plt.gca().set_ylabel('time [ticks]', fontsize=15);

## task ID per person versus time

In [ ]:
person_time_taskID_dict = {}
for person_id, person in enumerate(list_of_people):
    person_time_taskID_dict[person_id] = []
    for tick, work_dict in person.work_journal_per_tick.items():
        #print(work_dict)
        person_time_taskID_dict[person_id].append(work_dict['task']['task ID'])

In [ ]:
df_person_time_taskID = pandas.DataFrame(person_time_taskID_dict)
df_person_time_taskID

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

seaborn.heatmap(df_person_time_taskID)
plt.title('task ID')
plt.gca().set_xlabel('person', fontsize=15);
plt.gca().set_ylabel('time [ticks]', fontsize=15);

# ratio of the elapsed time for a task versus the number task for task.

In [ ]:
min(df_person_time_taskID.min())

In [ ]:
elapsed_task_timing = {}
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html
for task_id in range(min(df_person_time_taskID.min()),
                     max(df_person_time_taskID.max())):
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
    list_of_ticks_containing_task = list(df_person_time_taskID[
        df_person_time_taskID==task_id].dropna(axis=0,how='all').index)
    
    #assert(task_id==list_of_tasks[task_id-1]['task ID'])
    
    elapsed_task_timing[task_id] = {'first seen': min(list_of_ticks_containing_task),
                                    'last seen': max(list_of_ticks_containing_task),
                                    'task duration': 1}

In [ ]:
df_task_timing = pandas.DataFrame(elapsed_task_timing).T
df_task_timing['task actual duration'] = df_task_timing['last seen'] - df_task_timing['first seen']+1
df_task_timing['ratio of actual to minimum'] = df_task_timing['task actual duration']/df_task_timing['task duration']
#df_task_timing

In [ ]:
print("when the temporal cost of coordination is the same as just doing the work,")
print("with",number_of_people,"people the workload took",
      round(sum(df_task_timing['task actual duration'])/sum(df_task_timing['task duration']),2))
print("longer than if one qualified person had been assigned the same workload")

In [ ]:
# TODO: Heat map with axes of time and person index, 
# with the color being the status (working, idle, coordinating)

In [ ]:
# TODO: A pie chart of the named persona types like unicorn and specialist. 
# The biggest piece of pie will be the others

## task throughput vs time

In [ ]:
task_count_per_time_dict = {}
for person_id, person in enumerate(list_of_people):
    for tick, work_dict in person.work_journal_per_tick.items():
        if tick not in task_count_per_time_dict.keys():
            task_count_per_time_dict[tick] = 0
            
        if "task from" in work_dict.keys():
            #print(person_id, tick, work_dict["task from"])
            task_count_per_time_dict[tick] += 1

In [ ]:
plt.scatter(x=list(task_count_per_time_dict.keys()), 
            y=list(numpy.cumsum(list(task_count_per_time_dict.values()))),
            label="number of people="+str(number_of_people))

plt.ylabel('cumulative task count')
plt.xlabel('time [ticks]');
#plt.legend();

# The reward for good work is more work

In [ ]:
people_versus_time_task_count={}
for person_id, person in enumerate(list_of_people):
    people_versus_time_task_count[person_id] = []
    completed_task_count = 0
    for tick, work_dict in person.work_journal_per_tick.items():
        if work_dict['outcome']=='task completed':
            completed_task_count+=1
        people_versus_time_task_count[person_id].append(completed_task_count)

In [ ]:
list_of_labels = []
list_of_task_counts = []

for person_index, completed_task_count in people_versus_time_task_count.items():
    description=""
    for specialization,skilllevel in list_of_people[person_index].skill_specialization_dict.items():
        description+=specialization+str(skilllevel)+","

    person_label = "person "+str(person_index)+": "+description[:-1]
    
    # these two lists are not used in this cell; they are for the stacked plot
    list_of_labels.append(person_label)
    list_of_task_counts.append(completed_task_count)
        
    plt.scatter(x=range(len(completed_task_count)),
                y=completed_task_count, 
                label=person_label)
    
# https://stackoverflow.com/a/4701285
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('time [ticks]')
plt.ylabel('cumulative count\nof completed tasks');
plt.title('social circle size ='+str(social_circle_size));

# TODO: https://en.wikipedia.org/wiki/Little%27s_law